## Created by: Naufal Daffa Abdurahman

# 2. Import Library

In [24]:
# Data Manipulation and Analysis
import pandas as pd
import numpy as np

# Machine Learning Models
from sklearn.ensemble import RandomForestRegressor

# Saving and Loading Models
import pickle

#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform

In [25]:
new_test_data = pd.DataFrame({
    "HallwayType": ["terraced", "corridor"],
    "TimeToSubway": ["0-5min", "15min~20min"],
    "SubwayStation": ["Kyungbuk_uni_hospital","Sin-nam"],
    "N_FacilitiesNearBy(ETC)": [5.0, 0.0],
    "N_FacilitiesNearBy(PublicOffice)": [7.0, 1.0],
    "N_SchoolNearBy(University)": [5.0, 2.0],
    "N_Parkinglot(Basement)": [1321.0, 0.0],
    "YearBuilt": [2014, 1986],
    "N_FacilitiesInApt": [10, 4],
    "Size(sqf)": [1795, 500]
})
new_test_data

,HallwayType,TimeToSubway,SubwayStation,N_FacilitiesNearBy(ETC),N_FacilitiesNearBy(PublicOffice),N_SchoolNearBy(University),N_Parkinglot(Basement),YearBuilt,N_FacilitiesInApt,Size(sqf)
0,terraced,0-5min,Kyungbuk_uni_hospital,5.0,7.0,5.0,1321.0,2014,10,1795
1,corridor,15min~20min,Sin-nam,0.0,1.0,2.0,0.0,1986,4,500


# Data Cleaning (new_test_data)

In [26]:
# Change data type from float to integer
new_test_data['N_FacilitiesNearBy(ETC)'] = new_test_data['N_FacilitiesNearBy(ETC)'].astype('int64')
new_test_data['N_FacilitiesNearBy(PublicOffice)'] = new_test_data['N_FacilitiesNearBy(PublicOffice)'].astype('int64')
new_test_data['N_SchoolNearBy(University)'] = new_test_data['N_SchoolNearBy(University)'].astype('int64')
new_test_data['N_Parkinglot(Basement)'] = new_test_data['N_Parkinglot(Basement)'].astype('int64')

In [27]:
# rename category in 'TimeToSubway'
new_test_data.loc[new_test_data['TimeToSubway'] == '0-5min', 'TimeToSubway']='0~5min'
new_test_data.loc[new_test_data['TimeToSubway'] == 'no_bus_stop_nearby', 'TimeToSubway']='no_subway_nearby'

# Setting Variable Need for Google Cloud

In [28]:
## set up authentication using services account 
import os
# Authenticate using service account
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/home/ndaffaabdurahman/daffa-021/sa-development.json"

# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../vertex-ai-ml/dti-ds-31329ac0651d.json"

project_id = 'dti-ds'
dataset_id = 'daffa_dataset_021'
table_id = 'test_kaggle_file_no_target'
region = 'us-central1'
bucket_name = 'daffa_gcs_021'
blob_name = 'data/test_kaggle_file_no_target.csv'

model_name = 'rf_model.sav'

# Google Cloud Storage

## Retrieve from Google Cloud Storage

In [29]:
# Retrieve the model from Google Cloud Storage
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name) # Add bucket name
    blob_model = bucket.blob(f'model/{model_name}')
    blob_model.download_to_filename('rf_model.sav')

    print ("Read model succeeded")
except:
    raise TypeError("An exception occurred")

Read model succeeded


In [30]:
# Retrieve the dataset from Google Cloud Storage
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name) # Add bucket name
    blob_model = bucket.blob(blob_name)
    blob_model.download_to_filename('test_kaggle_file_no_target.csv')

    print ("Read Data succeeded")
except:
    raise TypeError("An exception occurred")

Read Data succeeded


# Model Prediction

In [31]:
# Load the model
with open('rf_model.sav', 'rb') as f:
    loaded_model = pickle.load(f)

y_pred_cloud_new_data = loaded_model.predict(new_test_data)
y_pred_cloud_new_data

array([410427.85869076,  57857.95152454])

In [32]:
new_test_cloud = new_test_data.copy()
new_test_cloud['SalePrice Prediction'] = y_pred_cloud_new_data
new_test_cloud

,HallwayType,TimeToSubway,SubwayStation,N_FacilitiesNearBy(ETC),N_FacilitiesNearBy(PublicOffice),N_SchoolNearBy(University),N_Parkinglot(Basement),YearBuilt,N_FacilitiesInApt,Size(sqf),SalePrice Prediction
0,terraced,0~5min,Kyungbuk_uni_hospital,5,7,5,1321,2014,10,1795,410427.858691
1,corridor,15min~20min,Sin-nam,0,1,2,0,1986,4,500,57857.951525


# Data Cleaning (unseen data)

In [33]:
# rename category in 'TimeToSubway'
auto_cloud.loc[auto_cloud['TimeToSubway'] == '0-5min', 'TimeToSubway']='0~5min'
auto_cloud.loc[auto_cloud['TimeToSubway'] == 'no_bus_stop_nearby', 'TimeToSubway']='no_subway_nearby'

In [40]:
# Renaming columns that contain '(' and replacing them with '_'
auto_cloud.columns = auto_cloud.columns.str.replace(r'\(', '_').str.replace(r')', '')

In [34]:
# loaded_model.predict(X_test.iloc[3:13])
y_pred_file_cloud = loaded_model.predict(auto_cloud.sample(frac=1))
y_pred_file_cloud[:13]

array([225928.81391301, 310229.71502519, 159093.36448749, 149578.21630001,
       213810.81400878,  52928.84082903, 215493.66928462, 257436.40896325,
        99890.59536958, 183474.97852183, 213810.81400878, 196573.89349337,
       384334.92294358])

In [41]:
auto_cloud

,HallwayType,TimeToSubway,SubwayStation,N_FacilitiesNearBy_ETC,N_FacilitiesNearBy_PublicOffice,N_SchoolNearBy_University,N_Parkinglot_Basement,YearBuilt,N_FacilitiesInApt,Size_sqf
0,corridor,0~5min,Kyungbuk_uni_hospital,0.0,3.0,2.0,0.0,1978,3,903
1,corridor,0~5min,Kyungbuk_uni_hospital,0.0,3.0,2.0,0.0,1978,3,903
2,corridor,0~5min,Kyungbuk_uni_hospital,0.0,3.0,2.0,0.0,1978,3,903
3,corridor,0~5min,Kyungbuk_uni_hospital,0.0,3.0,2.0,0.0,1978,3,903
4,corridor,0~5min,Kyungbuk_uni_hospital,0.0,3.0,2.0,0.0,1978,3,903
...,...,...,...,...,...,...,...,...,...,...
1763,mixed,15min~20min,Myung-duk,1.0,5.0,4.0,79.0,1992,3,644
1764,mixed,15min~20min,Myung-duk,1.0,5.0,4.0,79.0,1992,3,644
1765,mixed,15min~20min,Myung-duk,1.0,5.0,4.0,79.0,1992,3,644
1766,mixed,15min~20min,Myung-duk,1.0,5.0,4.0,79.0,1992,3,644


# Send to BigQuery

In [42]:
from google.cloud import bigquery

table_id = 'unseen_test_kaggle_file_no_target' 

# Construct a BigQuery client object.
client = bigquery.Client()

# Define the full table ID
table_full_id = f"{client.project}.{dataset_id}.{table_id}"

# Define the table schema (you could skip this part, but make sure for column naming w/o empty spaces)
# Make sure the column name without "space" so change it to underscore "_" or just without any space
# schema = [
#     bigquery.SchemaField("Vehicle Class", "STRING"),
#     bigquery.SchemaField("Coverage", "STRING"),
#     bigquery.SchemaField("Renew_Offer_Type", "STRING"),
#     bigquery.SchemaField("EmploymentStatus", "STRING"),
#     bigquery.SchemaField("MaritalStatus", "STRING"),
#     bigquery.SchemaField("Education", "STRING"),
#     bigquery.SchemaField("Number_of_Policies", "FLOAT"),
#     bigquery.SchemaField("MonthlyPremiumAuto", "FLOAT"),
#     bigquery.SchemaField("TotalClaimAmount", "FLOAT"),
#     bigquery.SchemaField("Income", "FLOAT"),
#     bigquery.SchemaField("CLVPrediction", "FLOAT")
# ]

auto_cloud.columns = ['_'.join(i.split(' ')) for i in auto_cloud.columns] # make sure no empty space on columns' name

# Create the table
# table = bigquery.Table(table_full_id)
# table = client.create_table(table, exists_ok=True)
# print(f"Created table {table.project}.{table.dataset_id}.{table.table_id}")

# Load the DataFrame into the BigQuery table
job = client.load_table_from_dataframe(auto_cloud, table_full_id) # your df predicted name, and table full id above

# Wait for the job to complete
job.result()
print(f"Loaded {job.output_rows} rows into {table_full_id}")

Loaded 1768 rows into dti-ds.daffa_dataset_021.unseen_test_kaggle_file_no_target
